In [1]:
#Import Libraries

#for reading and data-manipulation
import os
import numpy as np
import pandas as pd

In [2]:
#for visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from wordcloud import WordCloud
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [3]:
#for data preprocessing
import time
from contractions import contractions_dict
import re
from collections import Counter
from wordcloud import STOPWORDS
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
# for ML model Implementation
from gensim import corpora
from gensim.models import CoherenceModel, LdaModel
from gensim.models.coherencemodel import CoherenceModel
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ParameterGrid
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
import hdbscan
from sentence_transformers import SentenceTransformer
from umap import UMAP

/Users/binitkc/ADS_FiNAL/ADS/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
# This code avoids printing different warnings in following jupyter cells.
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)
# warnings.simplefilter("ignore", category=SettingWithCopyWarning)


In [6]:
df = pd.read_csv('lenovoK8.csv', encoding='ISO-8859-1') # this is for the LenovoK8 dataset.
df.head()

,sentiment,review
0,1,Good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...
3,1,Good
4,0,The worst phone everThey have changed the last...


In [7]:
df = df.rename(columns={'review': 'brief_review'})


In [8]:

df = df[['brief_review', 'sentiment']].dropna()


In [9]:
# df = df_filtered.rename(columns={'article_id': 'unique_identifier', 'content': 'brief_review'})

In [10]:
# df = df[['unique_identifier', 'brief_review', 'category']].dropna()

In [11]:
# Handling missing values
df.isnull().sum()

brief_review    0
sentiment       0
dtype: int64

In [12]:
#handling duplicate values 
len(df[df.duplicated(subset=['brief_review'])])

1708

In [13]:
# Drop duplicate 
df_cleaned = df.drop_duplicates(subset = ["brief_review"])
print(df_cleaned.head())

                                        brief_review  sentiment
0             Good but need updates and improvements          1
1  Worst mobile i have bought ever, Battery is dr...          0
2  when I will get my 10% cash back.... its alrea...          1
3                                               Good          1
4  The worst phone everThey have changed the last...          0


In [14]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12967 entries, 0 to 14674
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brief_review  12967 non-null  object
 1   sentiment     12967 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 303.9+ KB


In [15]:
# Expand Contraction

# Function to expand contractions using the contractions_dict
def expand_contractions(text):
    # Regular expression pattern to match contractions
    contractions_pattern = re.compile('({})'.format('|'.join(contractions_dict.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)

    def expand_match(contraction):
        match = contraction.group(0)
        expanded = contractions_dict.get(match.lower())
        return expanded

    expanded_text = contractions_pattern.sub(expand_match, text)
    return expanded_text

# Apply the expand_contractions function to the "brief_review" column
df_cleaned['brief_review'] = df_cleaned['brief_review'].apply(expand_contractions)

In [16]:
# Remove Punctuations

# Function to remove punctuations from text
def remove_punctuations(text):
    # Create a translation table to remove punctuations
    translator = str.maketrans('', '', string.punctuation +'\n')

    # Apply the translation table to remove punctuations
    text_without_punctuations = text.translate(translator)
    return text_without_punctuations

# Apply the remove_punctuations function to the "brief_review" column
df_cleaned['brief_review'] = df_cleaned['brief_review'].apply(remove_punctuations)

In [17]:
# Convert text in the "brief_review" column to lowercase
df_cleaned['brief_review'] = df_cleaned['brief_review'].str.lower()

In [18]:
# Function to remove URLs from text
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return re.sub(url_pattern, '', text)

# Function to remove words containing digits from text
def remove_words_with_digits(text):
    return ' '.join(word for word in text.split() if not any(char.isdigit() for char in word))

# Function to remove non-ASCII characters (special characters)
def remove_special_characters(text):
    # Replace non-ASCII characters with a space
    return re.sub(r'[^\x00-\x7F]+', ' ', text)

# Apply the remove_urls function to the "brief_review" column
df_cleaned['brief_review'] = df_cleaned['brief_review'].apply(remove_urls)

# Apply the remove_words_with_digits function to the "brief_review" column
df_cleaned['brief_review'] = df_cleaned['brief_review'].apply(remove_words_with_digits)

# Apply the remove_special_characters function to the "brief_review" column
df_cleaned['brief_review'] = df_cleaned['brief_review'].apply(remove_special_characters)

# Verify the cleaned data
df_cleaned['brief_review'].head()


0               good but need updates and improvements
1    worst mobile i have bought ever battery is dra...
2         when i will get my cash back its already ary
3                                                 good
4    the worst phone everthey have changed the last...
Name: brief_review, dtype: object

In [19]:
 # Remove Stopwords
# Download the list of stopwords if not already downloaded
nltk.download('stopwords')

# Get the list of English stopwords
stop_words = set(stopwords.words('english'))

# Add unnecessary words to the list of English stopwords
stop_unnecessary_words = stop_words.union(set(['mr', 'people', 'would', 'year', 'said', 'say', 'also', 'wale', 'could']))


# Function to remove stopwords from text
def remove_stopwords(text):
    words = text.split()
    words = [word for word in words if len(word)>2]
    filtered_words = [word for word in words if word.lower() not in stop_unnecessary_words]
    return ' '.join(filtered_words)

# Apply the remove_stopwords function to the "brief_review" column
df_cleaned['brief_review'] = df_cleaned['brief_review'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/binitkc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
# Remove white spaces from the "brief_review" column
df_cleaned['brief_review'] = df_cleaned['brief_review'].str.replace('\s+', ' ', regex=True)

In [21]:
# Tokenization

# Download the punkt package
nltk.download('punkt')
# Download the punkt_tab resource as well
nltk.download('punkt_tab')

# Function to tokenize text
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

# Apply the tokenize_text function to the "brief_review" column
df_cleaned['tokenized_content'] = df_cleaned['brief_review'].apply(tokenize_text)


[nltk_data] Downloading package punkt to /Users/binitkc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/binitkc/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [22]:
df_cleaned.head()

,brief_review,sentiment,tokenized_content
0,good need updates improvements,1,"[good, need, updates, improvements]"
1,worst mobile bought ever battery draining like...,0,"[worst, mobile, bought, ever, battery, drainin..."
2,get cash back already ary,1,"[get, cash, back, already, ary]"
3,good,1,[good]
4,worst phone everthey changed last phone proble...,0,"[worst, phone, everthey, changed, last, phone,..."


In [23]:
# Initialize the stemmer
stemmer = PorterStemmer()

# Function to perform stemming on text
def stem_text(tokens):
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens

# Apply the stem_text function to the "tokenized_content" column
df_cleaned['stemmed_content'] = df_cleaned['tokenized_content'].apply(stem_text)

In [24]:
nltk.download('wordnet')
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to perform lemmatization on text
def lemmatize_text(tokens):
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

# Apply the lemmatize_text function to the "tokenized_content" column
df_cleaned['lemmatized_content'] = df_cleaned['tokenized_content'].apply(lemmatize_text)

[nltk_data] Downloading package wordnet to /Users/binitkc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
df_cleaned['lemmatized_content'].info()

<class 'pandas.core.series.Series'>
Index: 12967 entries, 0 to 14674
Series name: lemmatized_content
Non-Null Count  Dtype 
--------------  ----- 
12967 non-null  object
dtypes: object(1)
memory usage: 202.6+ KB


In [26]:
df_cleaned = df_cleaned.iloc[0:10000]

In [27]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 0 to 11237
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   brief_review        10000 non-null  object
 1   sentiment           10000 non-null  int64 
 2   tokenized_content   10000 non-null  object
 3   stemmed_content     10000 non-null  object
 4   lemmatized_content  10000 non-null  object
dtypes: int64(1), object(4)
memory usage: 468.8+ KB


In [28]:
# ML Model - 1 Implementation

# Convert the list of lemmatized tokens into a list of lists
lemmatized_text = [tokens for tokens in df_cleaned['lemmatized_content']]

# Create a dictionary of terms with term frequency filtering
dictionary = corpora.Dictionary(lemmatized_text)
dictionary.filter_extremes(no_below=3, no_above=0.85)

# Fit the Algorithm

# Create a Gensim corpus
corpus_gensim = [dictionary.doc2bow(text) for text in lemmatized_text]

# Build LDA model
num_topics = 5
lda_model = LdaModel(corpus_gensim, num_topics=num_topics, id2word=dictionary, passes=20, random_state= 40)

print('\n')
print('\n')
# Calculate coherence score
coherence_model = CoherenceModel(model=lda_model, texts=lemmatized_text, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()

# Print coherence score
print(f"Coherence Score: {coherence_score:.4f}")


# Print topics and associated words
for topic_id, topic_words in lda_model.print_topics():
    print('\n')
    print(f"Topic {topic_id}: {topic_words}")

/Users/binitkc/ADS_FiNAL/ADS/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/binitkc/ADS_FiNAL/ADS/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/binitkc/ADS_FiNAL/ADS/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/binitkc/ADS_FiNAL/ADS/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with

Coherence Score: 0.5570


Topic 0: 0.063*"battery" + 0.033*"phone" + 0.026*"charging" + 0.019*"day" + 0.019*"product" + 0.017*"charge" + 0.016*"get" + 0.016*"hour" + 0.016*"fast" + 0.016*"time"


Topic 1: 0.042*"phone" + 0.026*"working" + 0.019*"call" + 0.018*"buy" + 0.018*"worst" + 0.013*"amazon" + 0.013*"network" + 0.013*"mobile" + 0.012*"product" + 0.012*"service"


Topic 2: 0.075*"good" + 0.074*"camera" + 0.039*"phone" + 0.032*"quality" + 0.030*"battery" + 0.019*"performance" + 0.012*"better" + 0.012*"note" + 0.010*"sound" + 0.010*"backup"


Topic 3: 0.048*"problem" + 0.043*"phone" + 0.041*"heating" + 0.032*"issue" + 0.031*"good" + 0.027*"money" + 0.024*"mobile" + 0.016*"screen" + 0.015*"waste" + 0.012*"like"


Topic 4: 0.087*"phone" + 0.065*"good" + 0.065*"product" + 0.065*"mobile" + 0.061*"nice" + 0.045*"best" + 0.043*"price" + 0.023*"awesome" + 0.018*"excellent" + 0.016*"great"


In [29]:
# import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import ParameterGrid
# from gensim.models import LdaModel
# from gensim.models.coherencemodel import CoherenceModel

# Define a grid of hyperparameters to search over
param_grid = {
    'num_topics': [3, 4, 5, 6, 7, 8],
    'passes': [10,15,20,25]
}

best_coherence_score = -1
best_lda_model = None
scores = []  # To store coherence scores
topics = []  # To store number of topics
passes_list = []  # To store passes for each model

# Perform grid search
for params in ParameterGrid(param_grid):
    lda_model = LdaModel(corpus=corpus_gensim, id2word=dictionary, **params)
    coherence_model = CoherenceModel(model=lda_model, texts=lemmatized_text, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    
    # Track best model
    if coherence_score > best_coherence_score:
        best_coherence_score = coherence_score
        best_lda_model = lda_model
    
    # Collect data for plotting
    scores.append(coherence_score)
    topics.append(params['num_topics'])
    passes_list.append(params['passes'])


/Users/binitkc/ADS_FiNAL/ADS/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/binitkc/ADS_FiNAL/ADS/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/binitkc/ADS_FiNAL/ADS/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/binitkc/ADS_FiNAL/ADS/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with

In [ ]:
# After grid search is completed and best_lda_model is chosen:
best_lda_model.save("Trained_models/lda_news.model")

# Optionally, print confirmation
print(" LDA_news model saved successfully! ")

In [ ]:
# Load the saved model
best_lda_model = LdaModel.load("Trained_models/lda_news.model")

# Optionally, print to confirm successful loading
print("LDA model loaded successfully!")

In [ ]:
# Plot coherence score in 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(topics, passes_list, scores, c=scores, cmap='viridis', marker='o')
ax.set_xlabel('Number of Topics')
ax.set_ylabel('Passes')
ax.set_zlabel('Coherence Score')
ax.set_title("Coherence Score for Different Hyperparameter Combinations")
plt.show()

# Print best hyperparameters and coherence score
print("Best Hyperparameters:", best_lda_model)
print(f"Best Coherence Score: {best_coherence_score:.4f}")

# Print topics and associated words for the best model
for topic_id, topic_words in best_lda_model.print_topics():
    print(f"\nTopic {topic_id}: {topic_words}")

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_topic_diversity(lda_model, num_topics):
    # Get the topic-word distributions from the LDA model
    topic_word_distributions = [lda_model.get_topics()[i] for i in range(num_topics)]
    
    # Calculate pairwise cosine similarities between topics
    similarities = cosine_similarity(topic_word_distributions)
    
    # Set diagonal elements to zero (similarity of a topic with itself should not be counted)
    np.fill_diagonal(similarities, 0)
    
    # Calculate topic diversity (1 - average cosine similarity)
    avg_similarity = np.mean(similarities)
    topic_diversity = 1 - avg_similarity
    
    return topic_diversity

# Assuming 'best_lda_model' is the trained model and 'num_topics' is the number of topics
topic_diversity = calculate_topic_diversity(best_lda_model, best_lda_model.num_topics)

print(f"Topic Diversity: {topic_diversity:.4f}")


In [ ]:
# Visualize the topics
vis_data = gensimvis.prepare(lda_model, corpus_gensim, dictionary)
pyLDAvis.display(vis_data)

In [ ]:
# import matplotlib.pyplot as plt

# Parameters
num_top_words = 15  # Number of top words to display for each topic
num_topics = lda_model.num_topics  # Total number of topics in the LDA model

# Define grid size for plotting (adjust based on the number of topics)
num_rows = 3
num_cols = 3

fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 8))

# Plot the top words for each topic
for topic_idx in range(num_topics):
    row = topic_idx // num_cols
    col = topic_idx % num_cols

    # Get the top words and weights for the current topic
    top_words = lda_model.show_topic(topic_idx, topn=num_top_words)
    words, weights = zip(*top_words)

    # Plot horizontal bar chart for the topic
    axes[row, col].barh(words, weights, color='orange')
    axes[row, col].set_title(f"Top {num_top_words} Words for Topic {topic_idx}")
    axes[row, col].set_xlabel("Word Importance")
    axes[row, col].invert_yaxis()  # Show the most important word at the top

# Remove empty subplots if the grid is larger than the number of topics
for i in range(num_topics, num_rows * num_cols):
    fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.show()
